# OSC Market Intelligence App - Component Testing

This notebook provides interactive testing of the main components of the Market Intelligence application.


## 1. Setup and Imports


In [ ]:
# Import required modules
import sys
from pathlib import Path

# Ensure src is in the path
project_root = Path.cwd()
sys.path.insert(0, str(project_root))

from src.config import DatabricksConfig, DatabaseConfig, OSC_COLORS, OSC_FONTS
from src.databricks_client import (
    get_workspace_client,
    call_endpoint,
    get_user_info,
    format_response,
)
from src.database import (
    init_database,
    create_conversation,
    add_message,
    update_message,
    get_conversation_messages,
    get_user_conversations,
)

print("✅ All imports successful!")


## 2. Test Configuration


In [ ]:
# Load configurations
databricks_config = DatabricksConfig.from_env()
database_config = DatabaseConfig.from_env()

print("Databricks Configuration:")
print(f"  Host: {databricks_config.host}")
print(f"  Endpoint: {databricks_config.endpoint_name}")
print()
print("Database Configuration:")
print(f"  Host: {database_config.host or 'Not configured'}")
print(f"  Database: {database_config.name}")
print(f"  Port: {database_config.port}")


In [ ]:
# Display OSC branding colors
print("OSC Brand Colors:")
for name, color in OSC_COLORS.items():
    print(f"  {name:15s}: {color}")

print("\nOSC Fonts:")
for name, font in OSC_FONTS.items():
    print(f"  {name:15s}: {font}")


## 3. Test Databricks Authentication


In [ ]:
# Initialize Databricks client
try:
    client = get_workspace_client()
    print("✅ Databricks client initialized successfully")
    
    # Get user info
    user_info = get_user_info(client)
    print(f"\nUser Information:")
    print(f"  User ID: {user_info['user_id']}")
    print(f"  Display Name: {user_info['display_name']}")
    print(f"  Active: {user_info['active']}")
except Exception as e:
    print(f"❌ Error initializing Databricks client: {e}")
    print("\nMake sure you have configured your Databricks credentials in .env file")


## 4. Test Endpoint Call

**Note:** This will make an actual call to your Databricks endpoint. Make sure it's running and accessible.


In [ ]:
# Test a simple question
test_question = "What are the key trends in the Canadian securities market?"

try:
    print(f"Sending question: {test_question}")
    print("\nCalling endpoint...")
    
    response = call_endpoint(
        client,
        databricks_config,
        test_question
    )
    
    print("\n✅ Response received!")
    print("\nRaw response:")
    print(response)
    
    # Format the response
    formatted_answer = format_response(response)
    print("\nFormatted answer:")
    print(formatted_answer)
    
except Exception as e:
    print(f"\n❌ Error calling endpoint: {e}")
    print("\nThis could be due to:")
    print("  - Endpoint not running")
    print("  - Incorrect endpoint name")
    print("  - Network connectivity issues")
    print("  - Authentication problems")


## 5. Test Database Operations

**Note:** This requires a configured PostgreSQL/Lakebase database. Skip if not configured.


In [ ]:
# Check if database is configured
db_configured = bool(database_config.host and database_config.user)

if db_configured:
    print("✅ Database is configured")
    print("\nInitializing database schema...")
    
    try:
        init_database(database_config)
        print("✅ Database schema initialized successfully")
    except Exception as e:
        print(f"❌ Error initializing database: {e}")
        db_configured = False
else:
    print("⚠️ Database is not configured")
    print("Configure database credentials in .env to test database operations")


In [ ]:
# Test conversation creation and message storage
if db_configured:
    try:
        # Create a test conversation
        test_user_id = "test_user@example.com"
        conversation_id = create_conversation(database_config, test_user_id)
        print(f"✅ Created conversation with ID: {conversation_id}")
        
        # Add a test message
        message_id = add_message(
            database_config,
            conversation_id=conversation_id,
            user_id=test_user_id,
            question="What is the market outlook for 2024?",
            answer="The market shows positive indicators...",
            status="complete"
        )
        print(f"✅ Added message with ID: {message_id}")
        
        # Retrieve messages
        messages = get_conversation_messages(database_config, conversation_id)
        print(f"\n✅ Retrieved {len(messages)} message(s)")
        
        for msg in messages:
            print(f"\nMessage ID: {msg['id']}")
            print(f"  Question: {msg['question']}")
            print(f"  Answer: {msg['answer']}")
            print(f"  Status: {msg['status']}")
            print(f"  Created: {msg['created_at']}")
        
        # Get user conversations
        conversations = get_user_conversations(database_config, test_user_id)
        print(f"\n✅ User has {len(conversations)} conversation(s)")
        
    except Exception as e:
        print(f"❌ Error testing database operations: {e}")
else:
    print("⚠️ Skipping database tests - database not configured")


## 6. Test Response Formatting


In [ ]:
# Test different response formats
test_responses = [
    {
        "name": "Predictions format",
        "response": {"predictions": "This is a test prediction response"}
    },
    {
        "name": "OpenAI format",
        "response": {
            "choices": [
                {"message": {"content": "This is an OpenAI-style response"}}
            ]
        }
    },
    {
        "name": "Answer format",
        "response": {"answer": "This is a simple answer response"}
    },
    {
        "name": "Custom format",
        "response": {"custom_field": "Custom response", "metadata": {"score": 0.95}}
    }
]

print("Testing response formatting:")
print("=" * 60)

for test in test_responses:
    formatted = format_response(test["response"])
    print(f"\n{test['name']}:")
    print(f"  Input: {test['response']}")
    print(f"  Output: {formatted}")


## 7. End-to-End Test

Test the complete flow: ask a question, get a response, and save to database.


In [ ]:
# Complete workflow test
if db_configured:
    try:
        print("Starting end-to-end test...\n")
        
        # 1. Create a new conversation
        user_id = user_info['user_id']
        conv_id = create_conversation(database_config, user_id)
        print(f"1. ✅ Created conversation: {conv_id}")
        
        # 2. Ask a question
        question = "What are the main regulatory priorities for 2024?"
        print(f"2. 📝 Question: {question}")
        
        # 3. Call the endpoint
        print("3. 🔄 Calling Databricks endpoint...")
        response = call_endpoint(client, databricks_config, question)
        answer = format_response(response)
        print(f"   ✅ Got response")
        
        # 4. Save to database
        msg_id = add_message(
            database_config,
            conversation_id=conv_id,
            user_id=user_id,
            question=question,
            answer=answer,
            status="complete"
        )
        print(f"4. ✅ Saved message: {msg_id}")
        
        # 5. Retrieve and display
        messages = get_conversation_messages(database_config, conv_id)
        print(f"5. ✅ Retrieved conversation history")
        
        print("\n" + "=" * 60)
        print("CONVERSATION SUMMARY")
        print("=" * 60)
        for msg in messages:
            print(f"\n🤔 Question: {msg['question']}")
            print(f"\n💡 Answer: {msg['answer']}")
            print(f"\n📊 Status: {msg['status']}")
            print(f"⏰ Time: {msg['created_at']}")
        
        print("\n" + "=" * 60)
        print("✅ END-TO-END TEST SUCCESSFUL!")
        print("=" * 60)
        
    except Exception as e:
        print(f"\n❌ End-to-end test failed: {e}")
        import traceback
        traceback.print_exc()
else:
    print("⚠️ Skipping end-to-end test - requires database configuration")


## 8. Summary and Next Steps


In [ ]:
print("Component Testing Summary")
print("=" * 60)
print("\n✅ Completed Tests:")
print("   - Configuration loading")
print("   - OSC branding setup")
print("   - Databricks authentication")
print("   - User information retrieval")
print("   - Endpoint calling")
print("   - Response formatting")
if db_configured:
    print("   - Database operations")
    print("   - Conversation management")
    print("   - Message storage and retrieval")
    print("   - End-to-end workflow")
else:
    print("   ⚠️  Database tests skipped (not configured)")

print("\n📚 Next Steps:")
print("   1. Run the Streamlit app: uv run streamlit run app.py")
print("   2. Test the full UI with real questions")
print("   3. Configure database for conversation history (if not done)")
print("   4. Deploy to Databricks Apps for production use")
print("\n" + "=" * 60)


---

## Additional Testing

Use the cells below for ad-hoc testing and experimentation.


In [ ]:
# Scratch cell for custom testing
